# Synopsis

Frequently we are simply looking for specific words or phrases in a block of text and do not care about the rest of the text. However, sometimes we are interested in a pattern of text (such as a phone number), where the format is consistent but the actual text itself changes. In this unit, we will learn:

1. What a regular expression is
2. Available functions in the `re` package
3. How to identify and extract a text pattern in a large block of text.
4. How to develop and test regular expressions

# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from copy import copy, deepcopy
from pathlib import Path
from sys import path

path.append( str(Path.cwd().parent) )

In [ ]:
import re

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from collections import Counter
from random import random
from string import punctuation, whitespace

from Amaral_libraries.my_nlp_library import read_complete_works

In [ ]:
my_fontsize = 15

# Regular Expressions

[Regular expressions](https://en.wikipedia.org/wiki/Regular_expression) (or **regexes** in shorthand) are essentially a standalone scripting language.

This is good because it means you will be able to use what you learn in other contexts.


Regexes allow for searches of patterns that are not fixed but instead follow a particular set of rules.

Imagine that you are looking for Northwestern's Helpline in a document.  Then you would search for `847-491-4357` or `(847) 491-4357`, or maybe even for `1-4357`. Yes, it is getting complicated...

But what if you actually wanted to find **any** phone number in a document?

Let's say that we use `d` to represent any digit 0-9.  Then, we are looking for patterns of the form `ddd-ddd-dddd` or `(ddd) ddd-dddd`

Amazingly, regexes allow us to construct a generic compact text pattern that will then be matched through the entire text.  


## Regular expressions in Python

Methods for regular expressions in Python are implemented in the [`re` package](https://docs.python.org/3/library/re.html). There are a many functions, flags, and conventions.

A few basic functions that we will use are:

> `re.match()` : Determine if the RE matches at the beginning of the string.
>
>  `re.search()` : Scan through a string, looking for any location where this RE matches.
>
> `re.findall()` : Find all substrings where the RE matches, and returns them as a list.
>
> `re.finditer()` : Find all substrings where the RE matches, and returns them as an iterator object.


Some important conventions are:

> `|` stands for `or`
>
> `&` stands for `and`
>
> `.` stands for any character except a new line
>
> `^` stands for beginning of the string being searched
>
> `$` stands for end of the string or just before new line
>
> `\` allows for escaping special characters, i.e., search for character that are used in conventions 


## Let's check back on Othello

Now, let's go over an example so this is less abstract. 

In [ ]:
# Copied from previous notebook

complete_works, plays = read_complete_works()
    
print(len(complete_works))

plays

In [ ]:
title = 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE'
start_line = plays[title][1]
end_line = plays[title][2]

the_play = complete_works[start_line: end_line]

print(f"\nThe play Othello has {len(the_play)} lines.\n")

# Put it all into a single string

the_play = ' '.join(the_play)

print(f"\nThe play Othello has {len(the_play)} characters.\n")

In [ ]:
print(the_play[:500])

Ok, let's search for Othello

In [ ]:
print(re.match('Othello', the_play))
print()

print(re.match('OTHELLO', the_play))
print()


`re.match` finds no matches.  This is not surprising if you recall that it only attempts to match at the start of the string.



In [ ]:
print(re.findall('Othello', the_play))
print()


`re.findall` finds numerous matches.  This is not surprising since we expect the string Othello to appear frequently in the play.


In [ ]:
print(re.search('Othello', the_play))
print()

print(re.search('OTHELLO', the_play))
print()

`re.search` finds matches and the capitalized version finds a different match from the non-capitalized.  

Each returned a single match as a `re.Match` object.  This is actually quite cool because a match object has all sorts of attributes!

Let's look at them in detail.


In [ ]:
othello_match = re.search('Othello', the_play)
help(othello_match)


Great! `re.Match` objects always have a Boolean value of `True`.

They also have methods such as `.start()`, `.end()`, or `.span()`. 

Let's see what they are

In [ ]:
# Notice the print of a re.Match object already contains 
# important information
print(othello_match)
print()

print(othello_match.span())
print()

print(othello_match.group())
print()

print(othello_match.start())
print()

print(othello_match.end())
print()

Those numbers are locators in the string, which we can use to look at the surrounding text

In [ ]:
the_play[187: 194]

In [ ]:
the_play[othello_match.start()-20: othello_match.end()+20]

<br>

<br>





What about `re.finditer()`?

In [ ]:
print(re.finditer('Othello', the_play))
print()

Interesting! Notice the word `iterator` in there.  This suggests that `Python` is offering us a way to iterate through the results.

As you will recall, we can do this using a `for` loop.


In [ ]:
for i, match_item in enumerate(re.finditer('Othello', the_play)):
    print(f"{i:>3} -- {match_item}")

<br>

<br>

<br>

<br>


Nice! 

**We have a way to get the fullness of output of `re.search` but for all the matches**.

# Creating regular expressions

Now that we know what some `re` functions do, we are ready to start exploring the real power of the package.

Above, we where interested in finding **rigid patterns**. Let us know search for **flexible patterns**.



## Testing, testing, testing

Even though we have brought the matter up, in reality, we have not emphasized enough the need to create test for your code.

We were trying to cover the basics of the language and did not want to add another moving part to the learning process.

The analogy I may use is that it is much easier to learn to drive with an automatic transmission than with a manual.

Now we are switching to *manual transmission* because it is so crucial for writing robust code.



Above, we discussed phone numbers. However, those are too complicated as a starting point. Instead, we will start with searching for times.

Imagine you are fed-up with G..gle and A..le and what to write code to search your emails for times of appointments to add to your calendar. What would you do?

Times come in two major formats good (`hh:mm`, also called military) and bad (`+h:mm *`).

Consider the good system: 

> * The first `h` can take the values 0, 1, or 2 
> * The second `h` can take values 0-9
> * The first `m` can take values 0-5
> * The second `m` can take values 0-9

However, not all combinations are possible. For example, 27 is not acceptable for `hh`.

Consider the bad system:

> * The first `+` can be absent ot take the values 0 or 1 
> * The second `h` can take values 0-9
> * The first `m` can take values 0-5
> * The second `m` can take values 0-9
> * The * can take the values pm or am and there may be a space between m and * or maybe not 

In order to insure the correctness of our code, we should start by creating a list of examples that even though they are not matches look superficially correct and another list of examples that are correct.

Both lists should cover a broad range of possibilities.


In [ ]:
good_times = [' 03:43 ', ' 01:00 ', ' 12:59 ', 
              ' 13:00 ', ' 21:35 ']
not_good_times = ['orange', ' 03:60 ', ' 26:14 ', ' 0155 ', 
                  ' 21:355 ']

And it is helpful to have a testing function...

In [ ]:
def test_pattern(pattern, text, to_match = True):
    count = 0
    for item in text:
        match = re.search(pattern, item)
        if match:
            print(match)
            count += 1
        else:
            print(item)
            
    if to_match:
        print( f"\n----Matched {count} out of {len(text)}"
               f" possible matches.\n" )
    else:
        print(f"\n----Failed to match {len(text) - count} out of {len(text)}"
              f" non matches.\n")

    return

We will address this problem in a modular manner, so that we can clearly see what the granular operations are.

First, I define `hours_re` and `minutes_re` to store the expressions for matching hours and minutes respectively.

The simplest case for both is 00-09. Which means that the first digit is always 0 and then second digit is any number from 0 to 9. 

This is easily defined as `0[0-9]`


In [ ]:
hours_re = '(0[0-9])'

re_times = hours_re + ':' 
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

Not impressive, ah!

Let's include other possibilities such as 10 to 19


In [ ]:
hours_re = '(0[0-9]|1[0-9])'

re_times = hours_re + ':'
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

Almost there for hours. Let's add 20 to 23...

In [ ]:
hours_re = '(0[0-9]|1[0-9]|2[0-3])'

re_times = hours_re + ':'
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

The hours are looking pretty good. 

Let's add the minutes term. Any value from 00 to 59 works so `[0-5][0-9]`

In [ ]:
hours_re = '(0[0-9]|1[0-9]|2[0-3])'
minutes_re = '[0-5][0-9]'

re_times = hours_re + ':' + minutes_re
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

We are almost there. The problem is that while **21:355** is not a time, it contains **21:35** which is a time.

How to solve this? For it to be a time, it needs to have a something that is not a digit after the first **5**...

This [site](https://www.rexegg.com/regex-quickstart.html#chars) has a lot of good information to build regexes. If you look in there, you find the `\d` means any single digit -- which is equivalent to [0-9] -- and that `\D` means any single character that is not a digit.

Using these conventions, we can write

In [ ]:
hours_re = '(0\d|1\d|2[0-3])'
minutes_re = '[0-5]\d\D'

re_times = hours_re + ':' + minutes_re
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

This looks great. However, humans do not always follow rules precisely.  So one may find `3:27` instead if the correct `03:27`.  How to account for this?

**Yes, you do it!**

In [ ]:
hours_re = '(0\d|1\d|2[0-3])'
minutes_re = '[0-5]\d\D'

re_times = hours_re + ':' + minutes_re
print(f"Current re_string is:\n\t\t{re_times}\n")

test_pattern(re_times, good_times)            
test_pattern(re_times, not_good_times, False)

Awesome! You are now an expert and ready to move on to the case of bad_times.

I will start your list of examples, but you do the rest.

In [ ]:
bad_times = [' 03:43pm ', ' 01:00 AM ', ]
not_bad_times = ['orange', ' 03:50 XM ', ]

And you build the `re_string`

## Getting help, but testing the help you get

Times, dates and email addresses are common types of information that one wants to extract from documents.  Above we saw how to handle times.  You can look into dates as an exercise.

Let us consider email addresses now.  **[Wikipedia](https://en.wikipedia.org/wiki/Email_address) has a detailed explanation of the rules governing the construction of valid emails addresses that we transcribe here.**

### The `local-part`

How are email addresses constructed? The standard formulation is `local-part@domain`.  The `local-part`, if unquoted, may use any of these ASCII characters:

* uppercase and lowercase Latin letters A to Z and a to z
* digits 0 to 9
* printable characters !#$%&'*+-/=?^_`{|}~
* dot ., provided that it is not the first or last character and provided also that it does not appear consecutively (e.g., John..Doe@example.com is not allowed)

The maximum total length of the local-part of an email address is 64 octets, so in principle it can amount to 64 characters.


### The `domain`

The `domain` part of an email address has to conform to strict guidelines: it must match the requirements for a `hostname`, a list of dot-separated `DNS` labels, each label being limited to a length of 63 characters and consisting of:

* uppercase and lowercase Latin letters A to Z and a to z
* digits 0 to 9, provided that top-level domain names are not all-numeric
* hyphen -, provided that it is not the first or last character

Certain domains, for example those intended for documentation and testing, should not be resolvable and that as a result mail addressed to mailboxes in them and their sub-domains should be non-deliverable. Of note for e-mail are `example`, `invalid`, `example.com`, `example.net`, and `example.org`. 


Seems complicated. So, why don't we check whether someone already built a regex for email addresses.

If you search online, you may be able to find the following solution

> `^[a-zA-Z0-9.]+@[a-zA-Z0-9.]+.[a-zA-Z0-9]+$`

It appears appropriately complicated!  But how do we know whether it works?

Let's test it!

In [ ]:
emails = ['a@b.co', 'something@somethingelse.org', '89@42.info', 
          'something@something.else.com', ]
not_emails = ['@b.c', 'a@b.', 'something@somethingelse.', ]

In [ ]:
re_emails = '^[a-zA-Z0-9.]+@[a-zA-Z0-9.]+.[a-zA-Z0-9]+$'
print(f"Current re_string is:\n\t\t{re_emails}\n")

test_pattern(re_emails, emails)            
test_pattern(re_emails, not_emails, False)

Well it appears to work! 

At least it fits all are test cases... Should we create more examples to be sure?


<br>

<br>

<br>

<br>

<br>

<br>



Good decision.

For now, let's see if we can make sense of `re_email` by breaking it down in pieces as we did with re_times



In [ ]:
username_re = '^[a-zA-Z0-9.]+'
domain_re = '[a-zA-Z0-9.]+.[a-zA-Z0-9]+$'

re_emails = username_re + '@' + domain_re
print(f"Current re_string is:\n\t\t{re_emails}\n")

`username_re` has three parts:

> * `^` means what comes next has to be a the start of the string symbol. It only makes sense to add this if the string contains only the email and nothing else.
> * `[a-zA-Z0-9.]` means that the characters allowed include lower case letters, upper case letters, digits, and periods.
> * `+` means that the previous element must appear at least once
at the beginning means the string must start with the first expression. This is a very handy character when you care about words that are at the start of a line only. 

`domain_re` also has three parts:

> * `[a-zA-Z0-9.]` means that the characters allowed include lower case letters, upper case letters, digits, and periods.
> * `+` means that the previous element must appear at least once
> * `.[a-zA-Z0-9]+` means that after some characters that may include periods, there must come a period followed by at least one character that is a lower case letter, an upper case letters, or a digits.
> '$' means that the string ends here or at most has a new line. This also only makes sense if the string contains only the email and nothing else.

Breaking it down like this makes some issues with `re_emails` apparent


In [ ]:
emails = [' a@b.co ', ' something@somethingelse.org ', ' 89@42.info ', 
          ' something@something.else.com ', ]
not_emails = [' @b.c ', ' a@b. ', ' something@somethingelse. ', 
              ' '+'a'*1000+'@b.c ', ' a@b.c ', 
              ' a@b.ccccccccccccccccccccccccccccccccccc ',
              ' a@b.b.b.b.b.b.b.b.b.b.b.b.b.b.com ', ]

Our string is likely to contain more than just an email address.

The username cannot be of arbitrary length. 

There likely cannot be more than 4 or 5 intermediate levels before the final period.

The final string of the server cannot be a single character and cannot be longer than a few characters.

In [ ]:
# username is no longer than 256 characters
# \w covers digits, upper and lower case letters and _
#
username_re = '([\w.]{1,64})'

# domain server (xxx.xx) last portion must be 2-8 letters
# after which there must be some space
#
domain_re = '(\w{1,64}\.[a-zA-Z]{2,8}\s)'

# inner domain pieces (up to 4), each must end with .
inner_domain_re = '((\w{1,64}\.){0,4})'

re_emails = ( username_re + '@' + inner_domain_re + domain_re )

print(f"Current re_string is:\n\t\t{re_emails}\n")

test_pattern(re_emails, emails)            
test_pattern(re_emails, not_emails, False)

Excellent! It works in all of our new test cases!

We could continue to improve this regex (say by limiting the ending domain to only known domains).

**In order to keep readability, it is important that you try to break the `re string` into pieces that are individually easier to validate**.


# Playing with Jeb Bush's emails 

For those of you too young to know anything, this was a thing before the 2016 elections.

In 2015, before his ill-fated primary run for the Republican Party Presidential Nomination, Jeb Bush released a number of his e-mails in a bid for transparency. 

As usually happens, the release wasn't vetted very well and some constituents Social Security numbers were exposed. 

But let's ignore that and instead focus on finding who then FL Governor Jed Bush corresponded with. The data is located in the folder `Data/Emails/`.

Let's see what data we have there...

In [ ]:
emails_folder = Path.cwd() / 'Data' / 'Emails'

filenames = list( emails_folder.glob('*.txt') )
for i, filename in enumerate( filenames ):
    print(f"{i:>2} -- {filename}")


Let's pick the 8th file, from `01 Jan`.

Note that these files were encoded in the `ISO-8859-1` standard.


In [ ]:
with open( filenames[7], 'r', encoding = 'ISO-8859-1') as file_in:
    emails_string = file_in.read()
    
print(len(emails_string))
print()

print(emails_string[:300])


.


.


Clearly there are some differences to what we were looking at.

In particular, it seems that email may be enclosed by `<...>`.

That is handy and easy enough to incorporate into our `re_string`


In [ ]:
username_re = '([\w.]{1,64})'
domain_re = '(\w{1,64}\.[a-zA-Z]{2,8})'  # got rid of \s
inner_domain_re = '((\w{1,64}\.){0,4})'

re_emails = ( '<' + username_re + '@' + inner_domain_re + domain_re + '>' )

print(f"Current re_string is:\n\t\t{re_emails}\n")

for match in re.finditer(re_emails, emails_string[:5000]):
    print(match)

Note bad. All the matches are good email addresses.  

But could it be that we are missing something?

**Let's print the string to check!**

In [ ]:
print(emails_string[:5000])


Did you notice `jeb@jeb.org`? 

Clearly, not all emails are enclosed within `<...>`.

Let's correct that then


In [ ]:
username_re = '([\w.]{1,64})'
domain_re = '(\w{1,64}\.[a-zA-Z]{2,8})'  # got rid of \s
inner_domain_re = '((\w{1,64}\.){0,4})'

re_emails = ( username_re + '@' + inner_domain_re + domain_re )

print(f"Current re_string is:\n\t\t{re_emails}\n")

for match in re.finditer(re_emails, emails_string[:5000]):
    print(match)

Nice.

Do you need to buy an email list to sell your miracle COVID cure? I can offer you a very good deal!

Let's check how many there are in the entire file.


In [ ]:
username_re = '([\w.]{1,64})'
domain_re = '(\w{1,64}\.[a-zA-Z]{2,8})'  # got rid of \s
inner_domain_re = '((\w{1,64}\.){0,4})'

re_emails = ( username_re + '@' + inner_domain_re + domain_re )

print(f"Current re_string is:\n\n{re_emails}\n")

for i, match in enumerate( re.finditer(re_emails, emails_string[:]) ):
    print(f"{i:>5}--{match.group():>50} -- {match.start()}")

That's really a lot of compromised e-mail addresses. 

There are so many that we actually can reproduce some of the analysis we did with text but using emails as tokens instead of words.

Change the code so you store all the emails you found and repeat some of the analyses that you conducted earlier.



# Additional Resources

If you're interest in learning more about using and writing regular expression, you can continue with this documentation.

* [More Python documentation](https://docs.python.org/3/howto/regex.html#regex-howto)
* [A great little notebook](http://nbviewer.ipython.org/github/sampathweb/python_reference/blob/master/tutorials/useful_regex.ipynb)



# Exercises


If you look carefully, you will see that emails enclosed in `<...>` contain the name of the owner of the email account.

Can you associate names to emails?

How would you do it? One name to one email? One name to many emails (Jeb clearly used a bunch of different email addresses)?

If you use a `dictionary` to store the data, what would be the key? 

Besides names that appear close to the email address, you can also find names in the signature of the email. You can even find other information such as addresses and such.

Can you scrape that information?

People are clearly an important type of entity in a corpus of emails.  However, the emails themselves are important entities.

Can you isolate each email?

What **metadata** can you extract concerning a given email?

## Searching filesystems

Let's say that you want to find all `PDF` files in your home account in your computer.

How do you obtain the path of every single file? Glob-glob

How do you search each file path for whether the file is a `PDF` or not?

How can you confirm that the file is indeed a `PDF` file?



## Regex Golf

There are so many more complicated things you can do with regex, and there is even a game called [regex golf](http://regex.alf.nu) that the nerdiest of all nerds play from time to time where the object is to come up with the shortest way to match certain patterns while [avoiding others](http://nbviewer.ipython.org/url/norvig.com/ipython/xkcd1313.ipynb). This game can serve as good practice to improve your regular expression skills.

As a test, let's play a game of regex golf. Let's try to match Star Wars movie titles, but not Star Trek movie titles.

In [ ]:
# This was MUCH simpler then
#
starwars = [ 'The Phantom Menace', 'Attack of the Clones', 
             'Revenge of the Sith', 'A New Hope', 
             'The Empire Strikes Back', 'Return of the Jedi' ]

startrek = [ 'The Wrath of Khan', 'The Search for Spock', 
             'The Voyage Home', 'The Final Frontier', 
             'The Undiscovered Country', 'Generations',
             'First Contact', 'Insurrection', 'Nemesis']